In [1]:
%run "./A1 - Dynamic Connect 4.ipynb"


, , , , , , 
 , , , , , , 
O, ,X, , , , 
 , , ,O, , ,X
 , , , ,O,X,X
 , ,O, , ,O,X
 , , ,X,O, , 

[[ 0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0]
 [ 1  0 -1  0  0  0  0]
 [ 0  0  0  1  0  0 -1]
 [ 0  0  0  0  1 -1 -1]
 [ 0  0  1  0  0  1 -1]
 [ 0  0  0 -1  1  0  0]]

  , , , , , , 
  , , , , , , 
 O, ,X, , , , 
  , , ,O, , ,X
  , , , ,O,X,X
  , ,O, , ,O,X
  , , ,X,O, , 
  , , , , , , 
  , , , , , , 
 O, ,X, , , , 
  , , ,O, , ,X
  , , , ,O,X,X
  , ,O, , ,O,X
  , , ,X,O, , 
  , , , , , , 
  , , , , , , 
 O, ,X, , , , 
  , , ,O, , ,X
  , , , ,O,X,X
  , ,O, , ,O,X
  , , ,X,O, , 
  , , , , , , 
  , , , , , , 
 O, ,X, , , , 
  , , ,O, , ,X
  , , , ,O,X,X
  , ,O, , ,O,X
  , , ,X,O, , 
------- 37b1b09d1cfde25e10e35dabc809c733697f4223 --------
  , , , , , , 
  , , , , , , 
 O, ,X, , , , 
  , , ,O, , ,X
  , , , ,O,X,X
  , ,O, , ,O,X
  , , ,X,O, , 
------- 12bbf58d46b8e43134504d8944045cac66978083 --------
  , , , , , , 
  , , , , , , 
 O, ,X, , , , 
  , , , ,O, ,X
  , , , ,O,X,X
  , ,O, , ,O,X


In [44]:
def getAiMove(state, player, time_limit, tnServer):
    (bestScore, nextState) = alphaBetaAgent(put(state), player)
    moveArr = nextState-state
    moveStr = moveToStr(moveArr, player)
    tnServer.write(bytes(moveStr + '\n', 'ascii'))
#     print('Wrote/Response', readRemote(tnServer), '\n')
    print('Wrote :',moveStr)
    return nextState

In [59]:
def readRemote(tnServer):
    resp = tnServer.read_until(bytes('\n', 'ascii')).decode('ascii')
    print('Reading : ', resp)
    return resp

def getRemoteMove(tnServer, state, player):
    remoteStr = readRemote(tnServer)
    if not re.match('^[1-7]{2}[NSWE]', remoteStr) : 
        print('Cannot parse to move :', remoteStr);
        return None
    
    # validate opponent is moving correct piece
    if state[int(remoteStr[0])-1, int(remoteStr[1])-1] != player:
        print('Attempting to move wrong piece :', remoteStr);
        return None

    newState = state + strToMove(remoteStr, player)
    if not isValidState(newState) :
        print('Invalid State from Server\n', remoteStr)
        return None
    return newState

In [60]:
def game_ai_vs_remote(SERVER=SERVER, stateFile='state_q1a.txt'):

    with open(stateFile, 'r') as file:
        stateStr = file.read()
    state = strToState(stateStr)

    tnServer = connectToServer(SERVER)
    
    time_limit = float(10000)
    moveNum = 1
    player = SERVER['colour']
    server_turn = player == 'black'
    if server_turn : 
        resp = tnServer.read_until(bytes('\n', 'ascii')).decode('ascii')
        print('ServerTurn - Reading : ', resp)
    curPlayer = PLAYER_IDS['num'][PLAYER_IDS['color'].index('white')]
    
    while True :
        print(stateToStr(state))
        print('----- Move : ', moveNum,' Player : ',curPlayer,' LocalTurn :', not(server_turn),' -------')
        
        start_time = time.time()
        
        if server_turn:
            nextState = getRemoteMove(tnServer, state, curPlayer)
            if nextState is None : continue
            state=nextState
        else:
            state=getAiMove(state, curPlayer, time_limit, tnServer)



        if scoreMaxConnected(curPlayer, state) == 4:
            print('======= Winner !! == Player :',curPlayer)
            print(stateToStr(state))
            return        

        curPlayer = -1*curPlayer
        server_turn = not server_turn
        moveNum += 1

In [61]:
SERVER = {
    'host' : 'localhost',
    'port' : 12345,
    'colour' : 'black',
    'game_id' : 'mytestgame',
}

In [63]:
game_ai_vs_remote(SERVER)

waiting for game : mytestgame
ServerTurn - Reading :   black

  , , , , , , 
  , , , , , , 
 O, ,X, , , , 
  , , ,O, , ,X
  , , , ,O,X,X
  , ,O, , ,O,X
  , , ,X,O, , 
----- Move :  1  Player :  1  LocalTurn : False  -------
Reading :  44W

  , , , , , , 
  , , , , , , 
 O, ,X, , , , 
  , , , ,O, ,X
  , , , ,O,X,X
  , ,O, , ,O,X
  , , ,X,O, , 
----- Move :  2  Player :  -1  LocalTurn : True  -------
Wrote : 33S
  , , , , , , 
  , , , , , , 
 O, , , , , , 
  , ,X, ,O, ,X
  , , , ,O,X,X
  , ,O, , ,O,X
  , , ,X,O, , 
----- Move :  3  Player :  1  LocalTurn : False  -------
Reading :  33S

Attempting to move wrong piece : 33S

  , , , , , , 
  , , , , , , 
 O, , , , , , 
  , ,X, ,O, ,X
  , , , ,O,X,X
  , ,O, , ,O,X
  , , ,X,O, , 
----- Move :  3  Player :  1  LocalTurn : False  -------
Reading :  31S

  , , , , , , 
  , , , , , , 
  , , , , , , 
 O, ,X, ,O, ,X
  , , , ,O,X,X
  , ,O, , ,O,X
  , , ,X,O, , 
----- Move :  4  Player :  -1  LocalTurn : True  -------
Wrote : 43S
  , , , , , , 
  ,